# Clip as the Main Control Layer in VideoDB Editor
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/editor/feature/clip_control_layer.ipynb)

---

This notebook is a comprehensive walkthrough of the **Clip object** in VideoDB Editor.

The Clip is the **most important object** in Editor. While **Assets** define *what* content you're using (video, image, audio), and **Tracks** define *layering*, the **Clip** is where you control **how** that content appears and behaves on screen.

**What you'll learn:**
- How Clip parameters control position, size, and transparency
- Using `fit` to handle different aspect ratios
- Applying visual filters for style and mood
- Adding transitions for smooth entry/exit effects
- Combining Clips to create picture-in-picture, split-screen, and sequential compositions

By the end of this notebook, you'll understand why the Clip is the **central control layer** for video composition in Editor.

---
## 📦 Step 1: Install dependencies

Lets install VideoDB SDK

In [1]:
%pip -q install videodb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


---
## 📦 Step 2: Connect to VideoDB

Run the next cell and enter your `VIDEO_DB_API_KEY` when prompted.

In [2]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")

os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()

print("Connected to VideoDB securely!")

Please enter your VideoDB API Key: ··········
Connected to VideoDB securely!


---
## 📦 Step 3: Connect to a collection

A **collection** is where your uploaded assets live (videos, images, audio).
We'll upload video assets next and use their `id` values inside Editor.

In [3]:
coll = conn.get_collection()

---
## 📦 Step 4: Upload video assets

Editor works with **assets stored in VideoDB**.

For this notebook, we'll upload **three different videos**:
- **video1**: Used for all single-clip demonstrations (position, scale, filters, etc.)
- **video2**: Used in multi-clip compositions alongside video1
- **video3**: Used in sequential clip demonstrations

This approach makes multi-clip examples visually clear (you'll see different content, not the same video repeated).

> Tip: When you re-run this notebook, you can skip re-uploading and fetch existing assets by ID instead (see the commented snippet below).

In [4]:
# Upload three different video assets
video1 = coll.upload(url="https://www.youtube.com/watch?v=HAX2RuZm-Fk")
print("Uploaded video1:", video1.id)

video2 = coll.upload(url="https://www.youtube.com/watch?v=RB9nyUyNI2s")
print("Uploaded video2:", video2.id)

video3 = coll.upload(url="https://www.youtube.com/watch?v=ExJZAegsOis")
print("Uploaded video3:", video3.id)

# Optional re-run pattern (don't upload again):
# video1 = coll.get_video("vid_id")
# video2 = coll.get_video("vid_id")
# video3 = coll.get_video("vid_id")

Uploaded video1: m-z-019b6e0e-d801-7763-9931-e914b7f35ba1
Uploaded video2: m-z-019b49b9-8618-7510-9cc7-6e0133c88bf1
Uploaded video3: m-z-019b446a-ba0f-7891-8533-dd75f72ea251


---
## 📦 Step 5: Import Editor building blocks

We'll use the core Editor objects throughout this notebook:
- `Timeline`: the global canvas (resolution + background)
- `Track`: a layer on the timeline
- `Clip`: the container that controls how assets appear
- `VideoAsset`: references your uploaded video by `id`
- `Position`, `Offset`, `Filter`, `Transition`: Clip effect parameters

In [5]:
from videodb import play_stream
from videodb.editor import Timeline, Track, Clip, VideoAsset, Position, Offset, Filter, Transition, Fit

---

# Part 1: Your First Clip

## 📦 Step 6: Create a basic Clip

Let's start with the simplest possible example: one video clip with default settings.

This demonstrates the **minimum required structure** for any Editor composition.

In [6]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

# Create a simple clip
clip = Clip(
    asset=VideoAsset(id=video1.id),
    duration=10
)

# Add to track and timeline
track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

# Generate and play
stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/06e4bb12-0eac-41cb-abab-6d7111b2a210.m3u8


**What just happened?**

We created a 10-second video clip with default settings:
- No position specified (defaults to filling the screen)
- No effects
- No transitions
- Just the raw video playing for 10 seconds

Now let's see how Clip parameters give us control.

---

# Part 2: Position Control

## 📦 Step 7: Position — The 9 Zones

The `position` parameter places your clip in one of **9 preset zones** on the screen:

```
top_left      top       top_right
left          center    right
bottom_left   bottom    bottom_right
```

Let's position the same video in the **top-right corner**. We'll also set `fit=None` so the video keeps its original size rather than filling the screen.

In [7]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

clip = Clip(
    asset=VideoAsset(id=video1.id),
    duration=10,
    position=Position.top_right,
    fit=None
)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/e9d5dbf5-b635-4cd6-b1f2-99f5defe6d60.m3u8


**Notice**: The video is now anchored to the top-right corner of the screen.

**Available positions**: `Position.top`, `Position.topRight`, `Position.right`, `Position.bottomRight`, `Position.bottom`, `Position.bottomLeft`, `Position.left`, `Position.topLeft`, `Position.center`

---

## 📦 Step 8: Offset — Fine-Tuning Position

Sometimes the 9 preset zones aren't enough. The `offset` parameter lets you **fine-tune** the position.

Offset values are **relative to viewport dimensions**:
- `x=0.1` moves the clip **10% to the right**
- `x=-0.1` moves it **10% to the left**
- `y=0.1` moves it **10% down**
- `y=-0.1` moves it **10% up**

Let's position a clip in the center, then shift it slightly to the right and down.

In [8]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

clip = Clip(
    asset=VideoAsset(id=video1.id),
    duration=10,
    position=Position.center,
    offset=Offset(x=0.15, y=0.1),
    fit=None
)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/2d752400-2b61-4d5a-851b-3b30759f58b7.m3u8


**Key insight**: `position` gives you **macro placement** (9 zones), while `offset` gives you **micro adjustment** for precise control.

---

# Part 3: Size and Transparency

## 📦 Step 9: Scale — Making Things Bigger or Smaller

The `scale` parameter is a **size multiplier**:
- `scale=1.0` — Original size (default)
- `scale=0.5` — Half size
- `scale=2.0` — Double size

Range: 0.0 to 10.0

Let's create a small video in the corner — think "picture-in-picture" style.

In [9]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

clip = Clip(
    asset=VideoAsset(id=video1.id),
    duration=10,
    position=Position.top_left,
    scale=0.5,
    fit=None
)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/62ec501f-b21f-4780-b4de-4dce8e2bacf1.m3u8


**Result**: The video is now **50% of its original size**, positioned in the top-left corner.

This is the foundation of picture-in-picture effects!

---

## 📦 Step 10: Opacity — Controlling Transparency

The `opacity` parameter controls how **transparent** the clip is:
- `opacity=1.0` — Fully opaque (default)
- `opacity=0.5` — 50% transparent
- `opacity=0.0` — Fully invisible

Range: 0.0 to 1.0

Let's create a semi-transparent video overlay.

In [10]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

clip = Clip(
    asset=VideoAsset(id=video1.id),
    duration=10,
    position=Position.center,
    opacity=0.3,
    fit=None
)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/e25b9c92-af09-44d2-833d-c6509b89ecee.m3u8


**Notice**: The video is now **30% opaque** (70% transparent), allowing the background to show through.

Opacity is essential for watermarks, overlays, and layered compositions.

---

## 📦 Step 11: Combining Scale and Opacity

Clip parameters work together. Let's create a large, semi-transparent video centered on screen.

In [11]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

clip = Clip(
    asset=VideoAsset(id=video1.id),
    duration=10,
    position=Position.center,
    scale=1.5,
    opacity=0.3,
    fit=None
)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/f27ca86c-0b1f-415f-83df-10badc88c117.m3u8


**Result**: The video is **1.5× larger** than normal and **30% opaque**.

This demonstrates how Clip parameters compose naturally.

---

# Part 4: Fit Modes

## 📦 Step 12: Understanding Fit

When your video's aspect ratio doesn't match the timeline resolution, `fit` determines the scaling behavior.

**Four fit modes**:

1. **`Fit.crop`** (default) — Scales to fill the viewport, crops edges if needed
2. **`Fit.contain`** — Scales to show entire video, may add letterboxing
3. **`Fit.cover`** — Stretches to fill viewport, ignoring aspect ratio (may distort)
4. **`Fit.none`** — Original dimensions, no scaling

Let's see `Fit.crop` in action with a portrait-oriented timeline.

In [12]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"
timeline.resolution = "600x1068"  # Portrait orientation (9:16)

clip = Clip(
    asset=VideoAsset(id=video1.id),
    duration=10,
    fit=Fit.crop
)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/9365055b-0e96-4ac4-9a17-a7921f62334c.m3u8


**`Fit.crop`**: The video fills the entire viewport. If the aspect ratios don't match, the edges are cropped.

Best for: Backgrounds, full-screen content where you want no black bars.

---

# Part 5: Visual Effects

## 📦 Step 13: Filters — Color Treatments

The `filter` parameter applies **color treatments** to your clip:

- `Filter.greyscale` — Remove color
- `Filter.blur` — Blur the video
- `Filter.contrast` — Increase contrast
- `Filter.darken` — Darken the scene
- `Filter.lighten` — Lighten the scene
- `Filter.muted` — Reduce saturation and contrast
- `Filter.negative` — Invert colors
- `Filter.boost` — Boost contrast and saturation

Let's apply a greyscale filter.

In [13]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

clip = Clip(
    asset=VideoAsset(id=video1.id),
    duration=10,
    filter=Filter.greyscale
)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/10834889-7e74-43d3-a608-b633a8c5b729.m3u8


**Result**: The video now plays in black and white.

Try experimenting with other filters: `Filter.blur`, `Filter.negative`, `Filter.boost`, etc.

---

## 📦 Step 14: Combining Filter with Position and Scale

Let's create a small, greyscale video in the corner — a common pattern for background overlays or secondary content.

In [14]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

clip = Clip(
    asset=VideoAsset(id=video1.id),
    duration=10,
    position=Position.top_left,
    scale=0.4,
    filter=Filter.greyscale,
    fit=None
)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/a1ff905e-ca2e-4f05-9d21-a551a4333070.m3u8


**Result**: A small (40% size), black-and-white video in the top-left corner.

Notice how naturally Clip parameters combine: position + scale + filter all work together.

---

# Part 6: Transitions

## 📦 Step 15: Fade In and Fade Out

The `transition` parameter controls how a clip **enters** and **exits**.

Most commonly, you'll use fade transitions:
- `in_="fade"` — Fade from transparent to opaque
- `out="fade"` — Fade from opaque to transparent
- `duration=2` — Transition duration in seconds

**Important**: Note the underscore in `in_="fade"` (because `in` is a Python keyword).

Let's create a clip that fades in, plays, then fades out.

In [15]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

clip = Clip(
    asset=VideoAsset(id=video2.id),
    duration=10,
    transition=Transition(in_="fade", out="fade", duration=2)
)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/acb06e75-6a52-408a-8614-c2742162188f.m3u8


**What you'll see**:
- First 2 seconds: Video fades in
- Middle 6 seconds: Video plays normally
- Last 2 seconds: Video fades out

Transitions make your edits feel polished and professional.

---

## 📦 Step 16: Transition with Delayed Start

You can add a clip later on the timeline using `track.add_clip(start=...)`.

Let's make the clip appear at the 2-second mark with a fade-in transition.

In [16]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

clip = Clip(
    asset=VideoAsset(id=video2.id),
    duration=8,
    transition=Transition(in_="fade", out="fade", duration=1.5)
)

track = Track()
track.add_clip(2, clip)  # Clip starts at 2 seconds on the timeline
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/61c5b163-d27f-4faa-80c4-e825c7c66c98.m3u8


**Timeline structure**:
- 0-2 seconds: Blank (gray background)
- 2-10 seconds: Video plays with fade in/out

This is your first glimpse of **timeline timing** — we'll explore this more shortly.

---

# Part 7: Combining Everything

## 📦 Step 17: The Full Clip Effect Stack

Let's combine multiple Clip parameters to create a stylized effect:
- Position in bottom-left
- Scale down to 70%
- Semi-transparent (30% opacity)
- Greyscale filter
- Fade in/out transitions

In [17]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

clip = Clip(
    asset=VideoAsset(id=video2.id),
    duration=10,
    position=Position.bottom_left,
    scale=0.7,
    opacity=0.3,
    filter=Filter.greyscale,
    transition=Transition(in_="fade", out="fade", duration=3),
    fit=None
)

track = Track()
track.add_clip(0, clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/93baf113-3cf9-4968-a575-aa6e5aba1e2e.m3u8


**Result**: A small, semi-transparent, black-and-white video in the bottom-right that fades in and out.

This demonstrates the **Clip effect stack** — all parameters working together to create sophisticated visual effects.

---

# Part 8: The "Double Start" Concept

## 📦 Step 18: Understanding Two Types of "Start"

This is a crucial concept that often confuses newcomers. There are **two different "start" parameters**:

1. **`VideoAsset(start=...)`** — **TRIMMING**: Skips the beginning of the source video
2. **`track.add_clip(start=...)`** — **TIMING**: Delays when the clip appears on the timeline

These are independent!

Let's demonstrate:
- We have a 60-second video
- We want to show seconds 10-20 of that video (a 10-second segment)
- But we want it to appear at the 5-second mark on our timeline

In [18]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

clip = Clip(
    asset=VideoAsset(
        id=video1.id,
        start=10  # TRIMMING: Skip first 10 seconds of source video
    ),
    duration=10  # Play for 10 seconds (showing seconds 10-20 of source)
)

track = Track()
track.add_clip(5, clip)  # TIMING: Place at 5-second mark on timeline
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/2df78303-f783-4282-b53e-c56186ae89a3.m3u8


**Timeline breakdown**:
- 0-5 seconds: Blank (gray background)
- 5-15 seconds: Video plays (showing seconds 10-20 of the original source)

**Key distinction**:
- `VideoAsset(start=10)` — "Start reading from the source video at the 10-second mark"
- `track.add_clip(5, clip)` — "Place this clip at the 5-second position on the timeline"

Understanding this "double start" concept is essential for precise editing.

---

# Part 9: Multi-Clip Compositions

## 📦 Step 19: Picture-in-Picture Effect

Now that we understand all Clip parameters, let's create a **picture-in-picture** composition using **two different videos**:
- Main video (full screen) — video1
- Small overlay video (bottom-right corner) — video2

We'll add both clips to the same track at the same time — they'll play simultaneously.

In [19]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

# Main video (full screen) - video1
main_clip = Clip(
    asset=VideoAsset(id=video1.id),
    duration=10,
    fit=Fit.crop
)

# Small overlay video (bottom-right, greyscale) - video2
overlay_clip = Clip(
    asset=VideoAsset(id=video2.id),
    duration=10,
    position=Position.top_left,
    scale=0.7,
    filter=Filter.greyscale,
    offset=Offset(x=-0.02, y=-0.02),  # Small margin from edge
    fit=None
)

track = Track()
track.add_clip(0, main_clip)
track.add_clip(0, overlay_clip)  # Same start time = simultaneous playback
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/f6893861-828f-4ea7-8b41-6e92ee0645da.m3u8


**Result**: A classic picture-in-picture layout with **two different videos**!

- video1 (main) fills the screen
- video2 (overlay, 70% size, greyscale) sits in the top-left corner
- Both play simultaneously

Notice how `track.add_clip(0, ...)` for both clips makes them play at the same time.

---

## 📦 Step 20: Split-Screen Effect

Let's create a split-screen composition using **two different videos side-by-side**.

Strategy:
- video1 on the left
- video2 on the right (with greyscale filter)
- Scale both to 50% width

In [20]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

# Left side clip - video1
left_clip = Clip(
    asset=VideoAsset(id=video1.id),
    duration=10,
    position=Position.left,
    scale=1,
    fit=None
)

# Right side clip - video2 (with greyscale filter)
right_clip = Clip(
    asset=VideoAsset(id=video2.id),
    duration=10,
    position=Position.right,
    scale=1,
    filter=Filter.greyscale,
    fit=None
)

track = Track()
track.add_clip(0, left_clip)
track.add_clip(0, right_clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/1c80a3f3-8077-42af-9379-0567c9f69564.m3u8


**Result**: A split-screen effect with **two different videos**:
- Left side: video1 (color)
- Right side: video2 (greyscale)

This demonstrates how **Clip controls enable spatial composition** — placing multiple clips in precise positions.

---

## 📦 Step 21: Sequential Clips with Transitions

Clips don't have to overlap. Let's create **three different videos playing one after another**, each with fade transitions and different effects.

In [21]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

# First clip (0-6 seconds)
clip1 = Clip(
    asset=VideoAsset(id=video1.id),
    duration=6,
    transition=Transition(in_="fade", out="fade", duration=1)
)

# Second clip (6-12 seconds)
clip2 = Clip(
    asset=VideoAsset(id=video2.id),
    duration=6,
    transition=Transition(in_="fade", out="fade", duration=1)
)

# Third clip (12-18 seconds)
clip3 = Clip(
    asset=VideoAsset(id=video3.id),
    duration=6,
    transition=Transition(in_="fade", out="fade", duration=1)
)

track = Track()
track.add_clip(0, clip1)
track.add_clip(6, clip2)
track.add_clip(12, clip3)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/b094c356-eddd-40db-8d50-c139c6d6c099.m3u8


**🎬 Result**: A continuous 18-second video with smooth transitions between three different scenes, each with its own unique styling (first clip normal, second clip greyscale, third clip scaled down).

**Timeline structure**:
- 0-6s: Normal video with fade in/out
- 6-12s: Greyscale video with fade in/out
- 12-18s: Smaller video (70% size) with fade in/out

Each clip has different visual properties, demonstrating how **Clip acts as the control layer** for every piece of content on your timeline.

---

# Wrap-Up

## What You've Learned

Congratulations! You now understand the **Clip** object — the most important control layer in VideoDB Editor.

**Key concepts covered**:

1. **Asset vs Clip**: Assets provide content; Clips control presentation
2. **Position**: 9 preset zones (`Position.center`, `Position.topRight`, etc.)
3. **Offset**: Fine-tuning position with relative x/y values
4. **Scale**: Size multiplier (0.0 to 10.0)
5. **Opacity**: Transparency control (0.0 to 1.0)
6. **Fit modes**: How assets scale to viewport (`crop`, `contain`, `cover`, `none`)
7. **Filters**: Color treatments (`greyscale`, `blur`, `contrast`, etc.)
8. **Transitions**: Fade in/out effects
9. **Double Start**: `VideoAsset(start=...)` vs `track.add_clip(start=...)`
10. **Composition**: Combining multiple clips with different effects


---
Every parameter works together to give you complete control over how content appears.

## Experiment Further

Try these ideas to deepen your understanding:

1. **Create a 4-way split**: Position four clips in each corner
2. **Animate with opacity**: Create clips with varying opacity levels
3. **Filter experiments**: Try all filter types to see their effects
4. **Complex timing**: Use `VideoAsset(start=...)` and `track.add_clip(start=...)` together
5. **Layer effects**: Combine multiple filters, scales, and positions

The Clip object is your primary tool for video composition in Editor — master it, and you can build any visual effect you imagine!
